In [ ]:
from earthformer.cuboid_transformer.cuboid_transformer import CuboidTransformerModel
import torch

# Optimized config for Landsat 3-timestep forecasting
landsat_config = {
    'input_shape': (3, 128, 128, 9),    # 3 input timesteps, 128x128, 9 Landsat bands
    'target_shape': (3, 128, 128, 1),   # 3 output timesteps
    
    # Small model for prototyping
    'base_units': 96,                    # Small but efficient
    'num_heads': 6,                      # Divisible by base_units
    'enc_depth': [2, 2],                 # 2-level hierarchy (sufficient for short sequences)
    'dec_depth': [1, 1],                 # Matching decoder depth
    
    # Dropout for better generalization during prototyping
    'attn_drop': 0.1,
    'proj_drop': 0.1,
    'ffn_drop': 0.1,
    
    # Global vectors for capturing Landsat scene patterns
    'num_global_vectors': 8,
    'use_dec_self_global': True,
    'use_dec_cross_global': True,
    
    # Optimized for satellite imagery
    'pos_embed_type': 't+hw',            # Separate temporal and spatial embeddings
    'use_relative_pos': True,            # Good for satellite spatial patterns
    'ffn_activation': 'gelu',            # Works well for vision tasks
    
    # Cuboid settings optimized for short temporal sequences
    'enc_cuboid_size': [(2, 4, 4), (2, 4, 4)],     # Small temporal cuboids for 3 timesteps
    'enc_cuboid_strategy': [('l', 'l', 'l'), ('d', 'd', 'd')],
    
    # Cross-attention settings for decoder
    'dec_cross_cuboid_hw': [(4, 4), (4, 4)],
    'dec_cross_n_temporal': [1, 2],      # Use 1-2 temporal frames for cross-attention
}

# Create model
model = CuboidTransformerModel(**landsat_config)
print(f"✓ Landsat model created! Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Test with dummy Landsat data
batch_size = 4  # You can use larger batches with 40GB VRAM
dummy_landsat = torch.randn(batch_size, 3, 128, 128, 9)
print(f"Input shape: {dummy_landsat.shape}")

# Forward pass test
with torch.no_grad():
    output = model(dummy_landsat)
    print(f"Output shape: {output.shape}")
    print("✓ Forward pass successful!")

# Memory usage estimate
def estimate_memory_usage(model, input_shape, batch_size=1):
    model.eval()
    dummy_input = torch.randn(batch_size, *input_shape)
    
    # Rough memory estimate
    param_memory = sum(p.numel() * 4 for p in model.parameters()) / 1e9  # GB
    input_memory = dummy_input.numel() * 4 / 1e9  # GB
    
    print(f"Estimated memory usage:")
    print(f"  Parameters: {param_memory:.2f} GB")
    print(f"  Input (batch={batch_size}): {input_memory:.2f} GB")
    print(f"  Activation estimate: ~{param_memory * 2:.2f} GB")
    print(f"  Total estimate: ~{param_memory * 3 + input_memory:.2f} GB")

estimate_memory_usage(model, (3, 128, 128, 9), batch_size=8)

In [1]:
import os
import torch
from model import LandsatLSTPredictor
from dataset import LandsatDataModule
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
import wandb

def train_landsat_model(
    dataset_root: str = "./Data/Dataset",
    batch_size: int = 4,
    max_epochs: int = 100,
    learning_rate: float = 1e-4,
    num_workers: int = 4,
    gpus: int = 1,
    precision: str = "32",  # Start with 32-bit precision for stability
    accumulate_grad_batches: int = 1,
    val_check_interval: float = 1.0,
    limit_train_batches: float = 1.0,
    limit_val_batches: float = 1.0,
    experiment_name: str = "landsat_lst_prediction",
    checkpoint_dir: str = "./checkpoints",
    log_dir: str = "./logs",
    wandb_project: str = "landsat-lst-forecasting",
    wandb_tags: list = None
):
    """
    Complete training pipeline for Landsat LST prediction
    
    Args:
        dataset_root: Path to preprocessed dataset with Cities_Tiles and DEM_2014_Tiles
        batch_size: Training batch size
        max_epochs: Maximum training epochs
        learning_rate: Initial learning rate
        num_workers: Number of data loading workers
        gpus: Number of GPUs to use
        precision: Training precision ('32', '16', or '16-mixed')
        accumulate_grad_batches: Gradient accumulation steps
        val_check_interval: Validation frequency
        limit_train_batches: Fraction of training data to use (for debugging)
        limit_val_batches: Fraction of validation data to use (for debugging)
        experiment_name: Name for logging
        checkpoint_dir: Directory to save checkpoints
        log_dir: Directory for logs
        wandb_project: Weights & Biases project name
        wandb_tags: List of tags for the experiment
    """
    
    # Set up default tags
    if wandb_tags is None:
        wandb_tags = ["landsat", "lstm-prediction", "earthformer"]
    
    # Create directories
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    
    # Verify tiled dataset exists
    dataset_path = os.path.join(dataset_root)
    cities_tiles = os.path.join(dataset_path, "Cities_Tiles")
    dem_tiles = os.path.join(dataset_path, "DEM_2014_Tiles")
    
    if not os.path.exists(cities_tiles):
        raise FileNotFoundError(f"Cities_Tiles directory not found at {cities_tiles}. Please run convert_to_tiles() first.")
    if not os.path.exists(dem_tiles):
        raise FileNotFoundError(f"DEM_2014_Tiles directory not found at {dem_tiles}. Please run convert_to_tiles() first.")
    
    print(f"✅ Found tiled dataset at {dataset_root}")
    
    # Initialize data module
    data_module = LandsatDataModule(
        dataset_root=dataset_root,
        batch_size=batch_size,
        num_workers=num_workers,
        sequence_length=3
    )
    
    # Test data module setup to catch issues early
    print("Testing data module setup...")
    try:
        data_module.setup("fit")
        train_loader = data_module.train_dataloader()
        val_loader = data_module.val_dataloader()
        
        print(f"✅ Training batches: {len(train_loader)}")
        print(f"✅ Validation batches: {len(val_loader)}")
        
        # Test one batch
        sample_batch = next(iter(train_loader))
        inputs, targets = sample_batch
        print(f"✅ Sample batch - Inputs: {inputs.shape}, Targets: {targets.shape}")
        
    except Exception as e:
        print(f"❌ Data module test failed: {e}")
        raise
    
    # Create comprehensive config for wandb
    config = {
        "batch_size": batch_size,
        "max_epochs": max_epochs,
        "learning_rate": learning_rate,
        "num_workers": num_workers,
        "precision": precision,
        "accumulate_grad_batches": accumulate_grad_batches,
        "val_check_interval": val_check_interval,
        "limit_train_batches": limit_train_batches,
        "limit_val_batches": limit_val_batches,
        "dataset_root": dataset_root,
        "model_type": "CuboidTransformer",
        "input_shape": [3, 128, 128, 9],
        "target_shape": [3, 128, 128, 1],
        "sequence_length": 3,
        "train_batches": len(train_loader),
        "val_batches": len(val_loader),
        "total_train_samples": len(train_loader) * batch_size,
        "total_val_samples": len(val_loader) * batch_size,
    }
    
    # Initialize Weights & Biases logger
    logger = WandbLogger(
        project=wandb_project,
        name=experiment_name,
        tags=wandb_tags,
        config=config,
        save_dir=log_dir,
        log_model=True,  # Log model checkpoints to wandb
    )
    
    # Initialize model
    model = LandsatLSTPredictor(
        learning_rate=learning_rate,
        weight_decay=1e-5,
        warmup_steps=1000,
        max_epochs=max_epochs
    )
    
    # Test model with sample data
    print("Testing model with sample data...")
    try:
        model.eval()
        with torch.no_grad():
            test_output = model(inputs)
            print(f"✅ Model test - Output shape: {test_output.shape}")
    except Exception as e:
        print(f"❌ Model test failed: {e}")
        raise
    
    # Callbacks
    checkpoint_callback = ModelCheckpoint(
        dirpath=checkpoint_dir,
        filename=f'{experiment_name}-{{epoch:02d}}-{{val_loss:.3f}}',
        save_top_k=3,
        monitor='val_loss',
        mode='min',
        save_last=True,
        verbose=True  # Added verbose for better feedback
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        mode='min',
        verbose=True
    )
    
    lr_monitor = LearningRateMonitor(logging_interval='epoch')
    
    # Trainer configuration
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator='gpu' if gpus > 0 else 'cpu',
        devices=gpus if gpus > 0 else None,
        # precision=precision,  # Re-enabled precision setting
        accumulate_grad_batches=accumulate_grad_batches,
        val_check_interval=val_check_interval,
        limit_train_batches=limit_train_batches,
        limit_val_batches=limit_val_batches,
        callbacks=[checkpoint_callback, early_stopping, lr_monitor],
        logger=logger,
        log_every_n_steps=50,
        enable_progress_bar=True,
        enable_model_summary=True,
        deterministic=False,  # Set to True for reproducibility, False for speed
        benchmark=True,  # Optimize for consistent input sizes
    )
    
    # Print comprehensive training info
    print(f"\n{'='*70}")
    print(f"LANDSAT LST PREDICTION TRAINING - TILED DATASET")
    print(f"{'='*70}")
    print(f"Dataset: {dataset_root}")
    print(f"  - Cities Tiles: {cities_tiles}")
    print(f"  - DEM Tiles: {dem_tiles}")
    print(f"Batch size: {batch_size}")
    print(f"Max epochs: {max_epochs}")
    print(f"Learning rate: {learning_rate}")
    print(f"Precision: {precision}")
    print(f"Devices: {gpus} GPU(s)" if gpus > 0 else "CPU")
    print(f"Num workers: {num_workers}")
    print(f"Experiment: {experiment_name}")
    print(f"Data limits: {limit_train_batches*100:.0f}% train, {limit_val_batches*100:.0f}% val")
    print(f"Checkpoints: {checkpoint_dir}")
    print(f"Logs: {log_dir}")
    print(f"Wandb project: {wandb_project}")
    print(f"Wandb tags: {wandb_tags}")
    print(f"{'='*70}\n")
    
    # Train the model
    try:
        print("🚀 Starting training...")
        trainer.fit(model, data_module)
        
        # Test the model if we have test data
        print("\n🧪 Running final test...")
        try:
            test_results = trainer.test(model, data_module, ckpt_path='best')
            print(f"✅ Test completed: {test_results}")
        except Exception as e:
            print(f"⚠️ Test failed (this is okay if no test data): {e}")
        
        print(f"\n🎉 Training completed successfully!")
        print(f"📁 Best model saved to: {checkpoint_callback.best_model_path}")
        print(f"🔗 View experiment at: {logger.experiment.url}")
        
        # Log final artifacts to wandb
        if checkpoint_callback.best_model_path:
            wandb.save(checkpoint_callback.best_model_path)
        
    except KeyboardInterrupt:
        print("\n⚠️ Training interrupted by user")
        print(f"📁 Last checkpoint saved to: {checkpoint_callback.last_model_path}")
        
    except Exception as e:
        print(f"\n❌ Training failed with error: {e}")
        import traceback
        traceback.print_exc()
        
        # Log the error to wandb
        if 'logger' in locals():
            wandb.log({"error": str(e)})
        
        raise
    
    finally:
        # Ensure wandb run is finished
        if 'logger' in locals():
            wandb.finish()
    
    return trainer, model, data_module


# Quick test/debug function with conservative settings
def debug_training(dataset_root: str = "./Data/Dataset"):
    """Quick debug run with small dataset fraction and conservative settings"""
    print("🔧 Running debug training with tiled dataset...")
    
    trainer, model, data_module = train_landsat_model(
        dataset_root=dataset_root,
        batch_size=2,
        max_epochs=3,
        learning_rate=1e-3,
        num_workers=0,  # Disable multiprocessing for debugging
        gpus=1,
        precision="32",  # Use 32-bit for stability
        limit_train_batches=0.1,  # Use only 10% of data
        limit_val_batches=0.1,
        experiment_name="debug_tiled_landsat",
        val_check_interval=0.5,  # Check validation twice per epoch
        wandb_project="landsat-debug",
        wandb_tags=["debug", "tiled", "quick-test"]
    )
    
    print("✅ Debug training completed!")


# Enhanced debug function with quarter dataset
def debug_with_enhanced_logging(dataset_root: str = "./Data/Dataset"):
    """Debug run using a quarter of the dataset with enhanced logging"""
    print("🔧 Running enhanced debug training with quarter dataset...")
    
    trainer, model, data_module = train_landsat_model(
        dataset_root=dataset_root,
        batch_size=4,
        max_epochs=5,
        learning_rate=1e-3,
        num_workers=2,  # Some multiprocessing for realistic testing
        gpus=1,
        precision="32",  # Use 32-bit for stability
        limit_train_batches=0.25,  # Use quarter of training data
        limit_val_batches=0.25,   # Use quarter of validation data
        experiment_name="enhanced_debug_quarter_dataset",
        val_check_interval=0.5,  # Check validation twice per epoch
        wandb_project="landsat-debug",
        wandb_tags=["enhanced-debug", "quarter-dataset", "realistic-test"]
    )
    
    print("✅ Enhanced debug training completed!")


# Even more minimal debug function
def minimal_debug_training(dataset_root: str = "./Data/Dataset"):
    """Minimal debug run with absolute minimum settings"""
    print("🔧 Running minimal debug training...")
    
    trainer, model, data_module = train_landsat_model(
        dataset_root=dataset_root,
        batch_size=1,  # Smallest possible batch
        max_epochs=1,  # Just one epoch
        learning_rate=1e-3,
        num_workers=0,  # No multiprocessing
        gpus=0,  # Use CPU to avoid GPU issues
        precision="32",  # Standard precision
        limit_train_batches=0.05,  # Use only 5% of data
        limit_val_batches=0.05,
        experiment_name="minimal_debug_tiled",
        val_check_interval=1.0,
        wandb_project="landsat-debug",
        wandb_tags=["minimal", "debug", "cpu"]
    )
    
    print("✅ Minimal debug training completed!")


# Full training configurations
def full_training_gpu(dataset_root: str = "./Data/Dataset"):
    """Full training with optimized GPU settings"""
    print("🚀 Starting full GPU training...")
    
    return train_landsat_model(
        dataset_root=dataset_root,
        batch_size=8,  # Larger batch for better GPU utilization
        max_epochs=50,
        learning_rate=2e-4,
        num_workers=4,
        gpus=1,
        precision="16-mixed",  # Mixed precision for speed
        experiment_name="landsat_full_training",
        val_check_interval=1.0,
        wandb_project="landsat-lst-forecasting",
        wandb_tags=["full-training", "production", "earthformer", "gpu"]
    )



if __name__ == "__main__":
    # For debugging with enhanced logging:
    debug_with_enhanced_logging()
    
    # For full training with enhanced logging:
    # full_training_with_enhanced_logging()
    
    # For dataset analysis only:
    # analyze_dataset_structure_only()
    
    # For exporting metadata to JSON:
    # export_dataset_metadata_to_json()

/root/miniconda3/envs/earthformer15/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


🔧 Running enhanced debug training with quarter dataset...
✅ Found tiled dataset at ./Data/Dataset
Testing data module setup...
train split: 86 cities, 1351 tile sequences
val split: 19 cities, 212 tile sequences
✅ Training batches: 338
✅ Validation batches: 53
✅ Sample batch - Inputs: torch.Size([4, 3, 128, 128, 9]), Targets: torch.Size([4, 3, 128, 128, 1])


wandb: Currently logged in as: jesus-guerrero (jesus-guerrero-ml) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/root/miniconda3/envs/earthformer15/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Model initialized with 9,036,109 parameters
Testing model with sample data...
✅ Model test - Output shape: torch.Size([4, 3, 128, 128, 1])


...

/root/miniconda3/envs/earthformer15/lib/python3.10/site-packages/pytorch_lightning/utilities/cloud_io.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(

✅ Test completed: []

🎉 Training completed successfully!
📁 Best model saved to: ./checkpoints/enhanced_debug_quarter_dataset-epoch=04-val_loss=1679.302.ckpt
🔗 View experiment at: https://wandb.ai/jesus-guerrero-ml/landsat-debug/runs/7rg8pmlv


/root/miniconda3/envs/earthformer15/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:153: UserWarning: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
  rank_zero_warn(


epoch,▁▁▁▁▃▃▃▃▃▅▅▅▅▅▆▆▆▆█████
lr-AdamW,▁▁▁▁▁
train_loss_epoch,█▆▅▃▁
train_loss_step,█▆▄▇▇▄▂▁
train_mae_epoch,█▇▅▃▁
train_mae_step,█▇▄█▇▄▂▁
train_temp_mae_scaled,█▇▅▃▁
train_temp_rmse_scaled,█▇▅▃▁
trainer/global_step,▁▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▇▇▇▇▇███
val_correlation,▁▂▄▇▇█████
val_loss,█▇▇▆▅▄▃▂▂▁


✅ Enhanced debug training completed!
